# به نام خدا
پروژه تحلیل شبکه‌های عصبی

در این پروژه، مدل‌های شبکه عصبی با استفاده از داده‌های مصنوعی مورد تحلیل و ارزیابی قرار می‌گیرند. مراحل مختلف پروژه شامل ایجاد داده، تقسیم داده به آموزش و آزمایش، آموزش مدل‌ها با یک و دو لایه پنهان، و مقایسه عملکرد مدل‌ها با استفاده از نمودار می‌باشد.

## مرحله 1: ایجاد مجموعه داده مصنوعی
در این بخش، مجموعه داده مصنوعی با سه ویژگی و متغیر هدف تولید می‌شود. نوع مسئله می‌تواند رگرسیون یا طبقه‌بندی باشد.

In [ ]:
import numpy as np
import pandas as pd

# ایجاد داده مصنوعی
def create_synthetic_data(task_type='regression', n_samples=500):
    np.random.seed(42)
    X = np.random.rand(n_samples, 3) * 10  # سه ویژگی مستقل
    if task_type == 'regression':
        y = 3 * X[:, 0] + 2 * X[:, 1] - X[:, 2] + np.random.randn(n_samples) * 2  # متغیر هدف پیوسته
    elif task_type == 'classification':
        y = (X[:, 0] + X[:, 1] - X[:, 2] > 15).astype(int)  # متغیر هدف طبقه‌بندی
    else:
        raise ValueError("Invalid task_type. Choose 'regression' or 'classification'.")
    return X, y

# تنظیم نوع مسئله (رگرسیون یا طبقه‌بندی)
task_type = 'classification'  # یا 'regression'
X, y = create_synthetic_data(task_type=task_type)

## مرحله 2: تقسیم داده‌ها به دو بخش آموزشی و آزمایشی
70 درصد داده‌ها برای آموزش و 30 درصد برای آزمایش در نظر گرفته می‌شوند.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## مرحله 3: ارزیابی اولیه با اعتبارسنجی 10 بخشی (10-fold Cross-Validation)
در این مرحله، داده‌ها به 10 بخش تقسیم شده و مدل‌ها به صورت متقاطع ارزیابی می‌شوند.

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)

## مرحله 4: آموزش مدل با یک لایه پنهان
مدل شبکه عصبی با مقادیر مختلف برای تعداد نورون‌ها در لایه پنهان ارزیابی می‌شود.

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import cross_val_score

hidden_layer_sizes = [(10,), (50,), (100,)]
results_one_layer = []

for h in hidden_layer_sizes:
    if task_type == 'regression':
        model = MLPRegressor(hidden_layer_sizes=h, max_iter=1000, random_state=42)
    else:
        model = MLPClassifier(hidden_layer_sizes=h, max_iter=1000, random_state=42)

    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy' if task_type == 'classification' else 'neg_mean_squared_error')
    mean_score = np.mean(cv_scores)
    results_one_layer.append((h, mean_score))

print("نتایج شبکه عصبی با یک لایه پنهان:")
for h, score in results_one_layer:
    print(f"ساختار لایه: {h}, میانگین امتیاز: {score:.4f}")

## مرحله 5: آموزش مدل با دو لایه پنهان
مدل شبکه عصبی با مقادیر مختلف برای تعداد نورون‌ها در دو لایه پنهان ارزیابی می‌شود.

In [ ]:
hidden_layer_sizes = [(10, 10), (50, 50), (100, 100)]
results_two_layers = []

for h in hidden_layer_sizes:
    if task_type == 'regression':
        model = MLPRegressor(hidden_layer_sizes=h, max_iter=1000, random_state=42)
    else:
        model = MLPClassifier(hidden_layer_sizes=h, max_iter=1000, random_state=42)

    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy' if task_type == 'classification' else 'neg_mean_squared_error')
    mean_score = np.mean(cv_scores)
    results_two_layers.append((h, mean_score))

print("\nنتایج شبکه عصبی با دو لایه پنهان:")
for h, score in results_two_layers:
    print(f"ساختار لایه‌ها: {h}, میانگین امتیاز: {score:.4f}")

## مرحله 6: ارزیابی نهایی روی داده‌های آزمایشی
بهترین مدل از بخش قبل برای پیش‌بینی داده‌های آزمایشی استفاده می‌شود و نتایج آن بررسی می‌گردد.

In [ ]:
best_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
best_model.fit(X_train, y_train)

if task_type == 'classification':
    predictions = best_model.predict(X_test)
    test_score = accuracy_score(y_test, predictions)
    print("\nدقت روی داده‌های آزمایشی:", test_score)
else:
    predictions = best_model.predict(X_test)
    test_score = mean_squared_error(y_test, predictions)
    print(f"\nخطای میانگین مربعات: {test_score:.4f}")

## مرحله 7: رسم نمودار مقایسه‌ای
نمودار مقایسه‌ای عملکرد مدل‌ها با یک و دو لایه پنهان ترسیم می‌شود.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar([str(h[0]) for h, _ in results_one_layer], [s for _, s in results_one_layer], label='یک لایه')
plt.bar([str(h[0]) for h, _ in results_two_layers], [s for _, s in results_two_layers], label='دو لایه')
plt.xlabel('ساختار لایه‌ها')
plt.ylabel('میانگین امتیاز')
plt.title('مقایسه عملکرد مدل‌ها')
plt.legend()
plt.show()